In [ ]:
!wget -O dataset.zip "https://huggingface.co/datasets/lehoangan02/nlp/resolve/main/dataset.zip?download=true"
!unzip -q dataset.zip

In [3]:
!pip install -U vllm --pre --extra-index-url https://wheels.vllm.ai/nightly



Looking in indexes: https://pypi.org/simple, https://wheels.vllm.ai/nightly
  Using cached https://wheels.vllm.ai/8f8fda261a620234fdeea338f44093d5d8072879/vllm-0.13.0rc2.dev79%2Bg8f8fda261-cp38-abi3-manylinux_2_31_x86_64.whl (474.9 MB)
  Using cached regex-2025.11.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached cachetools-6.2.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached psutil-7.1.3-cp36-abi3-manylinux2010_x86_64.manylinux_2_12_x86_64.manylinux_2_28_x86_64.whl.metadata (23 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached blake3-1.0.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.6 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached transformers-4.

In [7]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.4 MB/s eta 0:00:00a 0:00:01


Import

In [8]:
from vllm import LLM, SamplingParams
from vllm.model_executor.models.deepseek_ocr import NGramPerReqLogitsProcessor
from PIL import Image
import pandas as pd
from tqdm import tqdm
import os
from jiwer import cer, wer


Load llm

In [12]:
llm = LLM(
    model="unsloth/DeepSeek-OCR",   # <-- THIS IS THE BASELINE
    enable_prefix_caching=False,
    mm_processor_cache_gb=0,
    logits_processors=[NGramPerReqLogitsProcessor],
    gpu_memory_utilization=0.6
)


INFO 12-12 16:51:41 [utils.py:253] non-default args: {'enable_prefix_caching': False, 'gpu_memory_utilization': 0.6, 'disable_log_stats': True, 'mm_processor_cache_gb': 0, 'logits_processors': [<class 'vllm.model_executor.models.deepseek_ocr.NGramPerReqLogitsProcessor'>], 'model': 'unsloth/DeepSeek-OCR'}
INFO 12-12 16:51:42 [model.py:514] Resolved architecture: DeepseekOCRForCausalLM
WARNING 12-12 16:51:42 [model.py:1929] Your device 'NVIDIA T400 4GB' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 12-12 16:51:42 [model.py:1979] Casting torch.bfloat16 to torch.float16.
INFO 12-12 16:51:42 [model.py:1636] Using max model len 8192
INFO 12-12 16:51:42 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.


/home/lhan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:647: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


(EngineCore_DP0 pid=373202) INFO 12-12 16:51:52 [core.py:93] Initializing a V1 LLM engine (v0.13.0rc2.dev79+g8f8fda261) with config: model='unsloth/DeepSeek-OCR', speculative_config=None, tokenizer='unsloth/DeepSeek-OCR', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None, kv_cache_metrics=False, kv_cache_

(EngineCore_DP0 pid=373202) /home/lhan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.12/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=373202) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=373202)   warnings.warn(


(EngineCore_DP0 pid=373202) INFO 12-12 16:52:06 [cuda.py:412] Using FLASHINFER attention backend out of potential backends: ['FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION']
(EngineCore_DP0 pid=373202) INFO 12-12 16:52:06 [layer.py:372] Enabled separate cuda stream for MoE shared_experts
(EngineCore_DP0 pid=373202) ERROR 12-12 16:52:06 [gpu_model_runner.py:3655] Failed to load model - not enough GPU memory. Try lowering --gpu-memory-utilization to free memory for weights, increasing --tensor-parallel-size, or using --quantization. See https://docs.vllm.ai/en/latest/configuration/conserving_memory/ for more tips. (original error: CUDA out of memory. Tried to allocate 280.00 MiB. GPU 0 has a total capacity of 3.62 GiB of which 69.75 MiB is free. Including non-PyTorch memory, this process has 3.00 GiB memory in use. Of the allocated memory 2.92 GiB is allocated by PyTorch, and 21.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_C

(EngineCore_DP0 pid=373202) Process EngineCore_DP0:
(EngineCore_DP0 pid=373202) Traceback (most recent call last):
(EngineCore_DP0 pid=373202)   File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
(EngineCore_DP0 pid=373202)     self.run()
(EngineCore_DP0 pid=373202)   File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
(EngineCore_DP0 pid=373202)     self._target(*self._args, **self._kwargs)
(EngineCore_DP0 pid=373202)   File "/home/lhan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 870, in run_engine_core
(EngineCore_DP0 pid=373202)     raise e
(EngineCore_DP0 pid=373202)   File "/home/lhan/Documents/GitHub/DipSheetFyneTun/myenv/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 857, in run_engine_core
(EngineCore_DP0 pid=373202)     engine_core = EngineCoreProc(*args, **kwargs)
(EngineCore_DP0 pid=373202)                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(EngineCore_DP0 pid=373

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

Sampling paramters

In [ ]:
sampling_param = SamplingParams(
    temperature=0.0,
    max_tokens=8192,
    extra_args=dict(
        ngram_size=30,
        window_size=90,
        whitelist_token_ids={128821, 128822},  # <td> and </td>
    ),
    skip_special_tokens=False,
)


Run baseline

In [11]:
IMG_ROOT = "dataset/images"
TEST_CSV = "dataset/test.csv"
PROMPT = "<image>\nFree OCR."    # same prompt as notebook baseline

df = pd.read_csv(TEST_CSV)

preds = []
gts = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    rel = row["filepath"]
    gt = str(row["text"])
    img_path = os.path.join(IMG_ROOT, rel)

    image = Image.open(img_path).convert("RGB")

    model_input = [{
        "prompt": PROMPT,
        "multi_modal_data": {"image": image}
    }]

    out = llm.generate(model_input, sampling_param)
    pred = out[0].outputs[0].text

    preds.append(pred)
    gts.append(gt)

print("CER:", cer(gts, preds))
print("WER:", wer(gts, preds))


  0%|          | 5/2881 [06:36<63:21:59, 79.32s/it]


KeyboardInterrupt: 

ERROR 12-12 16:34:10 [core_client.py:600] Engine core proc EngineCore_DP0 died unexpectedly, shutting down client.
